In [56]:
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from datetime import datetime
from sklearn.svm import SVC
import operator
import googlefinance.client as gfc
from matplotlib import pyplot as plt

from sklearn import linear_model
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import normalize
import statsmodels.api as sm


import quandl

%matplotlib notebook

The goal of this notebook is to create time series regression models using some regression models as well as "Forest" models (both regression and deep learning versions). 

The features for this part of the research are going to be extracted using data freely available in Python via Quandl and pandas Yahoo finance support. 

- NASDAQ Composite
- Dow Jones Industrial Average
- Frankfurt DAX
- London FTSE-100
- Paris CAC 40
- Tokyo Nikkei-225
- Hong Kong Hang Seng
- Australia ASX-200

The features will be extracted from the major market indicators above will have the following format:

- Date : in days

- Open : price of the stock at the opening of the trading (in US dollars)
- High : highest price of the stock during the day (in US dollars)
- Low : lowest price of the stock during the trading day (in US dollars)
- Close : price of the stock at the closing of the trading (in US dollars)
- Volume : amount of stocks traded (in US dollars)
- Adj Close : price of the stock at the closing of the trading adjusted with dividends (in US dollars)

In [79]:
# Get googlefinance major global ETF data

period = "12M"

params = [
    {'q': "IXIC",
     'x': 'INDEXNASDAQ'
    },
    {'q': '.DJI',
      'x': 'INDEXDJX'  
    },
    {'q':'DAX',
     'x':'INDEXDB'   
    },
    {'q': 'UKX',
      'x':'INDEXFTSE'   
    },
    {'q': 'PX1',
      'x': 'INDEXEURO'   
    },
    {'q': 'NI225',
      'x': 'INDEXNIKKEI'  
    },
    {'q': 'HSI',
      'x': 'INDEXHANGSENG'  
    },
    {'q': 'XJO',
      'x':'INDEXASX'   
    },
    {'q': "SPY",
     'x': 'NYSEARCA'
    }
]

feats1 = gfc.get_closing_data(params, period)


quandl.ApiConfig.api_key = "a1LZ4iFxZS6mzZUsf28u"
feats3 = quandl.get("USTREASURY/YIELD",start_date='2016-10-05',
                    end_date='2017-09-09')

feats2 = quandl.get("USTREASURY/BILLRATES",start_date='2016-10-05',
                   end_date='2017-10-05')


x = feats1
y = feats2
z = feats3

cols = y.columns.append(z.columns)
# Ugly but useful. recursively merge the two data frames
for i in cols:
    try:
        x[i] = y[i]
    except KeyError:
        pass
    try:r
        x[i] = z[i]
    except KeyError:
        pass
x = x.dropna()
y = x.pop('.DJI')
X = x

IndentationError: unexpected indent (<ipython-input-79-efb1a4ee9ae5>, line 58)

In [61]:
x = feats1
y = feats2
z = feats3

,4 Wk Bank Discount Rate,4 Wk Coupon Equiv,13 Wk Bank Discount Rate,13 Wk Coupon Equiv,26 Wk Bank Discount Rate,26 Wk Coupon Equiv,52 Wk Bank Discount Rate,52 Wk Coupon Equiv
Date,,,,,,,,
2016-10-05,0.26,0.26,0.32,0.32,0.47,0.48,0.62,0.63
2016-10-06,0.25,0.25,0.32,0.32,0.45,0.46,0.62,0.63
2016-10-07,0.25,0.25,0.32,0.32,0.45,0.46,0.63,0.64
2016-10-11,0.26,0.26,0.35,0.36,0.47,0.48,0.68,0.69
2016-10-12,0.25,0.25,0.36,0.37,0.48,0.49,0.67,0.68


In [75]:
feats1['g'] = 0
feats2['g'] = 0

x.head()
y.head()

,4 Wk Bank Discount Rate,4 Wk Coupon Equiv,13 Wk Bank Discount Rate,13 Wk Coupon Equiv,26 Wk Bank Discount Rate,26 Wk Coupon Equiv,52 Wk Bank Discount Rate,52 Wk Coupon Equiv,g
Date,,,,,,,,,
2016-10-05,0.26,0.26,0.32,0.32,0.47,0.48,0.62,0.63,0
2016-10-06,0.25,0.25,0.32,0.32,0.45,0.46,0.62,0.63,0
2016-10-07,0.25,0.25,0.32,0.32,0.45,0.46,0.63,0.64,0
2016-10-11,0.26,0.26,0.35,0.36,0.47,0.48,0.68,0.69,0
2016-10-12,0.25,0.25,0.36,0.37,0.48,0.49,0.67,0.68,0


,IXIC,.DJI,DAX,UKX,PX1,NI225,HSI,XJO,SPY,g,4 Wk Bank Discount Rate,4 Wk Coupon Equiv,13 Wk Bank Discount Rate,13 Wk Coupon Equiv,26 Wk Bank Discount Rate,26 Wk Coupon Equiv,52 Wk Bank Discount Rate,52 Wk Coupon Equiv,g


In [4]:
# Predict shift_by dates away: e.g. shift_by=10 -> two weeks out
shift_by = 5


#Clean and shift the data
X = X.dropna()
y = y.shift(shift_by).dropna() 
X = X[:-shift_by]


# Normalize the features using sklearn normalize
# X = normalize(X)


ValueError: Found array with 0 sample(s) (shape=(0, 27)) while a minimum of 1 is required by the normalize function.

In [ ]:
y = y.pct_change().fillna(0)

In [ ]:
X.shape, y.shape

In [ ]:
# Use 95% to train of the 12 month data set

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
training_range = int(X.shape[0] * .95)
X_train = X[0:training_range]
y_train = y[0:training_range]

X_test = X[training_range: ]
y_test = y[training_range: ]

In [ ]:
y_train.head()

# Ordinary Least Square

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)
y_predicted = reg.predict(X_test)

fig, ax = plt.subplots()
ax.plot(y_test.values, label='Actual SPY')
ax.plot(y_predicted, label='Predicted SPY')
ax.set_title('OLS Prediction for SPY using 1 year period')
plt.legend()



reg.score(X_test, y_test)

In [ ]:
sY = y
sX = X
sX = sm.add_constant(sX)

model = sm.OLS(sY,sX)
results = model.fit()
results.summary2()

# Ridge Method

In [ ]:
ridge = linear_model.Ridge(alpha=1e-6)
ridge.fit(X_train, y_train)
y_ridge_predicted = ridge.predict(X_test)

fig, ax = plt.subplots()
ax.plot(y_test.values, label='Actual SPY')
ax.plot(y_ridge_predicted, label='Predicted SPY')
ax.set_title('RidgeCV Prediction for SPY using 1 year period')
plt.legend()

ridge.score(X_test, y_test)

# Lasso 

In [ ]:
lasso = linear_model.Lasso(alpha = 1e-3)
lasso.fit(X_train, y_train)

y_predicted = lasso.predict(X_test)

fig, ax = plt.subplots()
ax.plot(y_test.values, label='Actual SPY')
ax.plot(y_predicted, label='Predicted SPY')
ax.set_title('Lasso Prediction for SPY using 1 year period')
plt.legend()

lasso.score(X_test, y_test)

# Random Forest Regressor

In [ ]:
regr = RandomForestRegressor(n_estimators=40, criterion='mae')
regr.fit(X_train, y_train)
regr_predicted = regr.predict(X_test)
fig, ax = plt.subplots()
ax.plot(y_test.values, label='Actual SPY')
ax.plot(regr_predicted, label='Predicted SPY')
ax.set_title('Random Forest Regression for SPY using 1 year period')
plt.legend()

regr.score(X_test, y_test)

In [ ]:
y_test.shape

In [ ]:
regr_predicted[-1]

In [ ]:
y_test[-1]

In [ ]:
data = quandl.get("WIKI/SPY", trim_start = "2000-12-12", trim_end = "2014-12-30")


In [ ]:
import datetime
import pandas as pd
from pandas_datareader import data, wb

list = ["APLE","BHP"] 
#list = ["ASX:AMP","ASX:BHP"] 
df_all_stock = pd.DataFrame([])

start = datetime.datetime(2016, 1, 1)
end = datetime.datetime(2017, 1, 1)

for row in list:

    row = str(row)
    df_stock = data.DataReader(row, "google", start, end)
    df_all_stock = df_all_stock.append(df_stock)
    df_all_stock['code'] = row

df_all_stock

In [ ]:
gfc.get_closing_data??
